# CS/ECE/ISyE 524 - Spr 2018 - HW 2 - Solutions
### Prepared by: Laurent Lessard

---
## 1. Polyhedron modeling

We saw that the set of $x$ such that $Ax \le b$ where $A \in \mathbb{R}^{m\times n}$ and $b\in \mathbb{R}^m$
describes a polyhedron. For each polyhedron below, find a matrix $A$ and vector $b$ such that $Ax \le b$
describes the polyhedron. Hint: since each inequality describes a different face, $m$ should be equal to
the number of faces. Make sure the inequalities go the right way!

**a)** Cube centered at the origin with vertices at $(\pm 1,\pm 1,\pm 1)$.

**b)** Octahedron centered at the origin with vertices at $(\pm 1,0,0)$, $(0,\pm 1,0)$, $(0,0,\pm 1)$.

**Solution for the cube**
For the cube, each face is given by a constraint on just one variable. For example, we have the faces $x \le 1$, $x \ge -1$, and similarly for $y$ and $z$. We can also write them as box constraints: $-1\le x\le 1$, and similarly for $y$ and $z$. To convert to standard inequalities, we need to put all the variables on the same side with the inequality sign going the same way. In other words: $x \le 1$, $-x \le 1$, and so on. Converting into matrices and vectors, we have:
$$
\begin{bmatrix}
1 & 0 & 0 \\
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1 \\
0 & 0 & -1 \end{bmatrix}
\begin{bmatrix} x \\ y \\ z \end{bmatrix}
\le \begin{bmatrix} 1 \\ 1 \\ 1 \\ 1 \\ 1 \\ 1 \end{bmatrix}
\qquad\text{(for the cube)}
$$
Note that we can also rearrange the rows of $A$ (and correspondingly rearrange the rows of $b$) and the result doesn't change. This amounts to writing the six inequalities in a different order, which makes no difference.

**Solution for the octahedron**
For the octahedron, each face is a hyperplane with a normal vector that is $(\pm 1, \pm 1, \pm 1)$. So each of the 8 faces is a different combination of +'s and -'s. To figure out the $b$ vectors, we can go one face at a time. The halfspace corresponding to all positive coordinates has equation $x+y+z\le 1$. The face opposite it has equation $x+y+z \ge -1$, which we can rearrange to $-x-y-z\le 1$. Proceeding in a similar fashion for the six other faces, we obtain the equations:
$$
\begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & -1 \\
1 & -1 & 1 \\
1 & -1 & -1 \\
-1 & 1 & 1 \\
-1 & 1 & -1 \\
-1 & -1 & 1 \\
-1 & -1 & -1 
\end{bmatrix}
\begin{bmatrix} x \\ y \\ z \end{bmatrix}
\le \begin{bmatrix} 1 \\ 1 \\ 1 \\ 1 \\ 1 \\ 1 \\1 \\1\end{bmatrix}
\qquad\text{(for the octahedron)}$$

---
## 2. Standard form with equality constraints

Rather than using the standard LP form we saw in class, some prefer using a form where all variables are nonnegative, all constraints are equality constraints, and the cost function is a minimization. So a general LP would look like:

$$\begin{aligned}
\text{minimize} \qquad& c^Tx \\
\text{subject to:} \qquad& Ax = b \\
& x \ge 0\\
\end{aligned}$$

Consider the following LP:
$$\begin{aligned}
\text{maximize} \qquad& 3z_1 - z_2 \\
\text{subject to:} \qquad& -z_1 + 6z_2 - z_3 + z_4 \ge -3\\
& 7z_2 + z_4 = 5 \\
& z_3 + z_4 \le 2 \\
& -1 \le z_2 \le 5,\quad -1 \le z_3 \le 5,\quad -2 \le z_4 \le 2
\end{aligned}$$

**a)** Transform the above LP into the equality-constrained standard form of (1). What are A, b, c,
and x? Be sure to explain how the decision variables of your transformed LP relate to those of
the original LP 

**b)** Solve both versions of the LP using JuMP and show that you can recover the optimal z and
objective value by solving your transformed version of the LP.

We have omitted the details for part a. The solution to part b is given below.

In [7]:
# Solution of the original problem

using JuMP,Clp
m = Model(solver=ClpSolver())

@variable(m, z[1:4] )
@constraints(m, begin
    -1 <= z[2] <= 5
    -1 <= z[3] <= 5
    -2 <= z[4] <= 2
    -z[1] + 6z[2] - z[3] + z[4] >= -3
    z[3] + z[4] <= 2
    7z[2] + z[4] == 5
end)
@objective(m, Max, 3z[1] - z[2])

status = solve(m)
println(m)
println(status)
println()
for i = 1:4
    println("z[", i, "] = ", getvalue(z[i]) )
end
println("objective = ", getobjectivevalue(m) )

Max 3 z[1] - z[2]
Subject to
 -1 <= z[2] <= 5
 -1 <= z[3] <= 5
 -2 <= z[4] <= 2
 -z[1] + 6 z[2] - z[3] + z[4] >= -3
 z[3] + z[4] <= 2
 7 z[2] + z[4] == 5
 z[i] for all i in {1,2,3,4}

Optimal

z[1] = 8.571428571428571
z[2] = 0.42857142857142855
z[3] = -1.0
z[4] = 2.0
objective = 25.28571428571429


In [8]:
# Solution of the new version of the problem

using JuMP, Clp
m = Model(solver=ClpSolver())

@variable(m, x[1:9] >= 0 )
@constraints(m, begin
    x[4] + x[5] == 6
    x[2] + x[3] == 6
    x[6] + x[7] == 4
    x[1]-5x[2]+x[3]+(5/6)x[4]-(1/6)x[5]-(1/2)x[6]+(1/2)x[7]+x[8] == 3
    (35/6)x[2]-(7/6)x[3]+(1/2)x[6]-(1/2)x[7] == 5
    (5/6)x[4]-(1/6)x[5]+(1/2)x[6]-(1/2)x[7]+x[9] == 2
end)
@objective(m, Min, -3x[1] + (5/6)x[2] - (1/6)x[3] )

status = solve(m)
println(m)
println(status)
println()
println("z[1] = ", getvalue( x[1] ))
println("z[2] = ", getvalue( (5/6)x[2] - (1/6)x[3]) )
println("z[3] = ", getvalue( (5/6)x[4] - (1/6)x[5]) )
println("z[4] = ", getvalue( (1/2)x[6] - (1/2)x[7]) )
println("objective = ", -getobjectivevalue(m) )

Min -3 x[1] + 0.8333333333333333 x[2] - 0.16666666666666666 x[3]
Subject to
 x[4] + x[5] == 6
 x[2] + x[3] == 6
 x[6] + x[7] == 4
 x[1] - 5 x[2] + x[3] + 0.8333333333333333 x[4] - 0.16666666666666666 x[5] - 0.5 x[6] + 0.5 x[7] + x[8] == 3
 5.833333333333333 x[2] - 1.1666666666666665 x[3] + 0.5 x[6] - 0.5 x[7] == 5
 0.8333333333333333 x[4] - 0.16666666666666666 x[5] + 0.5 x[6] - 0.5 x[7] + x[9] == 2
 x[i] >= 0 for all i in {1,2,..,8,9}

Optimal

z[1] = 8.571428571428571
z[2] = 0.4285714285714286
z[3] = -1.0
z[4] = 2.0
objective = 25.28571428571429


In [9]:
# Solution of the new version of the problem (condensed form)

using JuMP, Clp
m = Model(solver=ClpSolver())

@variable(m, x[1:9] >= 0 )

A = [ 0   0    0    1    1    0    0    0    0
      0   1    1    0    0    0    0    0    0
      0   0    0    0    0    1    1    0    0
      1  -5    1   5/6 -1/6 -1/2  1/2   1    0
      0  35/6 -7/6  0    0   1/2 -1/2   0    0
      0   0    0   5/6 -1/6  1/2 -1/2   0    1 ]

b = [ 6; 6; 4; 3; 5; 2 ]

c = [ -3; 5/6; -1/6; 0; 0; 0; 0; 0; 0 ]

@constraint(m, A*x .== b)
@objective(m, Min, dot(c,x))

status = solve(m)
println(m)
println(status)
println()
println("z[1] = ", getvalue( x[1]) )
println("z[2] = ", getvalue( (5/6)x[2] - (1/6)x[3]) )
println("z[3] = ", getvalue( (5/6)x[4] - (1/6)x[5]) )
println("z[4] = ", getvalue( (1/2)x[6] - (1/2)x[7]) )
println("objective = ", -getobjectivevalue(m) )

Min -3 x[1] + 0.8333333333333334 x[2] - 0.16666666666666666 x[3]
Subject to
 x[4] + x[5] == 6
 x[2] + x[3] == 6
 x[6] + x[7] == 4
 x[1] - 5 x[2] + x[3] + 0.8333333333333334 x[4] - 0.16666666666666666 x[5] - 0.5 x[6] + 0.5 x[7] + x[8] == 3
 5.833333333333333 x[2] - 1.1666666666666667 x[3] + 0.5 x[6] - 0.5 x[7] == 5
 0.8333333333333334 x[4] - 0.16666666666666666 x[5] + 0.5 x[6] - 0.5 x[7] + x[9] == 2
 x[i] >= 0 for all i in {1,2,..,8,9}

Optimal

z[1] = 8.571428571428571
z[2] = 0.4285714285714286
z[3] = -1.0
z[4] = 2.0
objective = 25.28571428571429


---
## 3. Alloy blending

The company Steelco has received an order for 500 tonnes of steel to be used in shipbuilding.  The steel must have the following charactersitics:

| Chemical Element | Minimum Grade | Maximum Grade |
|------------------|---------------|---------------|
| Carbon (C)       | 2             | 3             |
| Copper (Cu)      | 0.4           | 0.6           |
| Manganese (Mn)   | 1.2           | 1.65          |

The company has seven different raw materials in stock that may be used for the production of this steel. The following table lists the grades, available amounts and prices for all materials:

| Raw Material | C%     | Cu%    | Mn%    | Availability in t | Cost in $/t  |
|--------------|--------|--------|--------|-------------------|--------------|
| Iron1        | 2.5    | 0      | 1.3    | 400               | 200          |
| Iron2        | 3      | 0      | 0.8    | 300               | 250          |
| Iron3        | 0      | 0.3    | 0      | 600               | 150          |
| Cu1          | 0      | 90     | 0      | 500               | 220          |
| Cu2          | 0      | 96     | 4      | 200               | 240          |
| Al1          | 0      | 0.4    | 1.2    | 300               | 200          |
| Al2          | 0      | 0.6    | 0      | 250               | 165          |

The objective is to determine the composition of the steel that minimizes the production cost.

In [1]:
# store all the data for the problem

raw = [ :iron1, :iron2, :iron3, :cu1, :cu2, :al1, :al2 ]

# composition (in percent) of [C, Cu, Mn]
composition = Dict(
:iron1 => [2.5, 0,  1.3],
:iron2 => [3,   0,  0.8],
:iron3 => [0,  0.3,  0 ],
:cu1   => [0,  90,   0 ],
:cu2   => [0,  96,   4 ],
:al1   => [0,  0.4, 1.2],
:al2   => [0,  0.6,  0 ])

# availability in tonnes
availability = Dict(
:iron1 => 400,
:iron2 => 300,
:iron3 => 600,
:cu1   => 500,
:cu2   => 200,
:al1   => 300,
:al2   => 250)

# cost in dollars per tonne
cost = Dict(
:iron1 => 200,
:iron2 => 250,
:iron3 => 150,
:cu1   => 220,
:cu2   => 240,
:al1   => 200,
:al2   => 165)

# minimum and maximum grade of [C, Cu, Mn]
MinGrade = [2, 0.4, 1.2]
MaxGrade = [3, 0.6, 1.65]
;

In [2]:
using JuMP,Clp

m = Model(solver=ClpSolver())
@variable(m, use[raw] >= 0)   # amount of each raw material to use (in tonnes)
@expression(m, production, sum(use))
@constraint(m, avail[i in raw], use[i] <= availability[i])
@constraint(m, sum(composition[i]*use[i] for i in raw) .>= MinGrade*production)
@constraint(m, sum(composition[i]*use[i] for i in raw) .<= MaxGrade*production)
@constraint(m, production >= 500)
@objective(m, Min, sum(cost[i]*use[i] for i in raw))
solve(m)
println(getvalue(use))
println("The cost will be \$", getobjectivevalue(m))

use: 1 dimensions:
[iron1] = 400.0
[iron2] = 0.0
[iron3] = 39.77630199231041
[  cu1] = 0.0
[  cu2] = 2.7612722824187346
[  al1] = 57.46242572527083
[  al2] = 0.0
The cost will be $98121.63579168123


## 4. Stigler's diet

True story! In 1945, American economist (and future Nobel laureate) George Stigler published a paper investigating the composition of an _optimal_ diet; minimizing total cost while meeting the recommended daily allowance (RDA) of several nutrients. To answer this question, Stigler tabulated a list of 77 foods and their nutrient content for 9 nutrients: calories, protein, calcium, iron, vitamin A, thiamine, riboflavin, niacin, and ascorbic acid. Here is what the first few rows and columns of his table looked like:

|						| Calories (1000)	| Protein (g)	| Calcium (g) 	| Iron (mg) | ... |
|-----------------------|-------------------|---------------|---------------|-----------|-----|
|Wheat Flour (Enriched) | 44.7				| 1411			| 2				| 365		| ... |
|Macaroni				| 11.6				| 418			| 0.7			| 54		| ... |
|Wheat Cereal (Enriched)| 11.8				| 377			| 14.4			| 175		| ... |
| ...			        | ...		        | ... 		    | ... 		    | ...	    | ... |


To make calculations easier, Stigler normalized his data so each row shows the nutrients contained in \$1's worth of the given food item. Back then, \$1 could buy you quite a lot!

When Stigler posed his diet problem, the simplex method had not yet been invented. In his paper, he wrote: "...the procedure is experimental because there does not appear to be any direct method of finding the minimum of a linear function subject to linear conditions." Nevertheless, through a combination of "trial and error, mathematical insight, and agility", he eventually arrived at a solution: a diet costing only \$39.93 per year. Though he confessed: "There is no reason to believe that the cheapest combination was found, for only a handful of the [many] possible combinations of commodities were examined."

**a)** Formulate Stigler's diet problem as an LP and solve it. To get you started, Stigler's original data is provided in `stigler.csv`, and the IJulia notebook `stigler.ipynb` imports the data and puts it into a convenient array format. How does your cheapest diet compare in annual cost to Stigler's? What foods make up your optimal diet?

**b)** Suppose we wanted to find the cheapest diet that was also vegetarian. How much would that cost per year, and what foods would be used?

In [10]:
using NamedArrays

# import Stigler's data set
raw = readcsv("stigler.csv");
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );

In [16]:
# Write as a linear program and solve!

using JuMP,Clp
m = Model(solver=ClpSolver())

@variable(m, x[foods] >= 0)
for j in nutrients
    @constraint(m, sum( data[i,j]*x[i] for i in foods ) >= lower[j] )
end
@objective(m, Min, sum(x))

solve(m)
println("The optimal (daily) diet is:")
xopt = getvalue(x)
for i in foods
    if xopt[i] > 1e-6
        println(i, ": ", xopt[i])
    end
end
println()
println("The cost per year is: \$", 365*getobjectivevalue(m))

The optimal (daily) diet is:
Wheat Flour (Enriched): 0.02951906167648827
Liver (Beef): 0.0018925572907052643
Cabbage: 0.011214435246144865
Spinach: 0.005007660466725203
Navy Beans, Dried: 0.061028563526693246

The cost per year is: $39.66173154546625


In [17]:
# to make it vegetarian, we definitely can't have Beef liver. So let's remove it and re-solve:
@constraint(m, x["Liver (Beef)"] == 0)
solve(m)
println("Optimal diet without Beef liver:")
xopt = getvalue(x)
for i in foods
    if xopt[i] > 1e-6
        println(i, ": ", xopt[i])
    end
end
println()
println("The cost per year is: \$", 365*getobjectivevalue(m))

Optimal diet without Beef liver:
Wheat Flour (Enriched): 0.035455581408887736
Evaporated Milk (can): 0.008591461668763588
Cabbage: 0.011249517312443502
Spinach: 0.005112832613199644
Navy Beans, Dried: 0.048628043573168446

The cost per year is: $39.79866435040896


**Comment:** simply excluding beef liver doesn't guarantee that the resulting solution will be vegetarian. But if it is, it's the optimal vegetarian diet. In general we can do this sequentially by removing non-vegetarian items from our solution and re-solving until we get a vegetarian diet. This will give us the same solution as if we had just gone through the entire list of foods, removed all non-vegetarian items, and solved the problem using the modified list of foods.